In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import pylab as pl


from DataGenerator import DataGenerator
from Models import Models
from LoadData import LoadData
from BaselineHelperFunctions import BaselineHelperFunctions
from RandomGridSearch import RandomGridSearch
from CustomCallback import CustomCallback
from MinMaxScalerFitter import MinMaxScalerFitter
from StandardScalerFitter import StandardScalerFitter

import umap
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

from matplotlib.colors import ListedColormap
%matplotlib inline
plt.rcParams["figure.figsize"]= (15,15)
helper = BaselineHelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})


In [2]:
def y_to_sequence(y):
    label_dict = {'explosion':0, 'earthquake':1, 'noise':2, 'induced':3}
    y_seq = np.empty((len(y), 1))
    for idx, label in enumerate(y):
        y_seq[idx] = label_dict.get(label[0])
        
    return y_seq

# Data Prep

In [3]:
csv_root = 'csv_folder_3_class'
loadData = LoadData(csv_root, isBalanced = True)
shuffle = False
full_ds, train_ds, val_ds, test_ds = loadData.getDatasets(shuffle = shuffle)

In [4]:
test = False
detrend = False
sample_rate = 0.8

In [5]:
scalerFitter = StandardScalerFitter(train_ds)
scaler = scalerFitter.fit_scaler(test = test, detrend = detrend)

In [6]:
X_train, y_train = scalerFitter.transform_subsample(train_ds, subsample_rate = sample_rate, shuffle = shuffle, detrend = detrend)
X_val, y_val = scalerFitter.transform_subsample(val_ds, subsample_rate = sample_rate, shuffle = shuffle, detrend = detrend)
#X_train, y_train = scalerFitter.subsample(train_ds)
#X_val, y_val = scalerFitter.subsample(val_ds)

n_samples_train, channels, timesteps = X_train.shape
n_samples_val, _, _ = X_val.shape
X_train = X_train.reshape((n_samples_train, channels*timesteps))
X_val = X_val.reshape((n_samples_val, channels*timesteps))

y_train = y_to_sequence(y_train)
y_val = y_to_sequence(y_val)

## UMAP

In [ ]:
seed = 55
reducer = umap.UMAP(low_memory = True, random_state = seed)
trans = reducer.fit(X_train)

In [ ]:
plt.scatter(trans.embedding_[:, 0], trans.embedding_[:, 1], s= 3, c=y_train, cmap='jet')
plt.title('Embedding of the training set by UMAP', fontsize=24);

#### KNN and SVC

In [ ]:
knn = KNeighborsClassifier().fit(trans.embedding_, y_train.ravel())
svc = SVC().fit(trans.embedding_, y_train.ravel())

In [ ]:
# Score:
svc.score(trans.transform(X_val), y_val), knn.score(trans.transform(X_val), y_val)

TSNE og MDS